Set-up
---

In [1]:
!git clone https://github.com/Gsus01/NuweOracle-AnalisisDeSentimiento.git repo

Cloning into 'repo'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 58 (delta 0), reused 13 (delta 0), pack-reused 44
Unpacking objects: 100% (58/58), 386.96 MiB | 12.24 MiB/s, done.
Filtering content: 100% (10/10), 417.96 MiB | 95.75 MiB/s, done.


In [3]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached datasets-2.11.0-py3-none-any.whl (468 kB)
  Using cached transformers-4.28.1-py3-none-any.whl (7.0 MB)
  Using cached multiprocess-0.70.14-py39-none-any.whl (132 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.6 MB/s eta 0:00:00
  Using cached aiohttp-3.8.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached xxhash-3.2.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (212 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 15.1 MB/s eta 0:00:00


Entrenamiento
---

In [30]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import numpy as np

raw_datasets = load_dataset("csv", data_files="repo/data/train.csv")
raw_datasets = raw_datasets['train'].train_test_split(.2)
raw_datasets['validation'] = raw_datasets['test']
temp = load_dataset("csv", data_files="repo/data/test.csv")
raw_datasets["test"] = temp["train"]
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
  return tokenizer(example["text"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
  columns=["attention_mask", "input_ids", "token_type_ids"],
  label_cols=["label"],
  shuffle=True,
  collate_fn=data_collator,
  batch_size=8,
)

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
  columns=["attention_mask", "input_ids", "token_type_ids"],
  label_cols=["label"],
  shuffle=True,
  collate_fn=data_collator,
  batch_size=8,
)

tf_test_dataset = tokenized_datasets["test"].to_tf_dataset(
  columns=["attention_mask", "input_ids", "token_type_ids"],
  shuffle=False,
  collate_fn=data_collator,
  batch_size=8,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/2412 [00:00<?, ? examples/s]

Map:   0%|          | 0/604 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [31]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = 8
num_epochs = 3
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
  initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler)

In [33]:
import tensorflow as tf

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=num_epochs)

Epoch 1/3
302/302 [==============================] - 118s 215ms/step - loss: 0.3699 - accuracy: 0.8391 - val_loss: 0.2426 - val_accuracy: 0.8841
Epoch 2/3
302/302 [==============================] - ETA: 0s - loss: 0.1222 - accuracy: 0.9590

In [ ]:
model.save_pretrained("model", max_shard_size="50MB")

Test
---

In [17]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("model")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [18]:
preds = model.predict(tf_test_dataset)["logits"]

95/95 [==============================] - 7s 41ms/step


In [19]:
class_preds = np.argmax(preds, axis=1)
print(preds.shape, class_preds.shape)

(754, 2) (754,)


In [20]:
target = {}
for idx, pred in enumerate(preds):
  target[str(idx)] = 0 if pred[0] > pred[1] else 1
predictions = { 'target': target }

In [21]:
import json

with open('predictions.json', 'w') as f:
  json.dump(predictions, f, indent=2)

Descarga
---

In [ ]:
!tar -czvf model.tar.gz model/

In [ ]:
from google.colab import files

files.download('model.tar.gz')